# Document

## インポート + 定数 + 出力モード
Pythonに元から備わっているrandom関数なら今後も大きな変更はないであろうという希望的観測の元採用.

スロットタイムはいったん$9[\mathrm{\mu s}]$とした.

出力モードを設定できるようにした.0では衝突時のみログが流れる.1だと全てのログを出力する.2では結果のみが出力される.関数の呼び出し時に指定できる.

In [ ]:
import random

# Slot time
SLOT_TIME = 9 * 10**(-6)

# Output modes
print_mode = {
    0: "Only Collision",
    1: "ALL",
    2: "No Output"
}

## User Classの宣言
PythonのクラスはCの構造体のメンバーに関数も持てるようにした感じ。

ここではUser Classが初期値として以下を持つ
- seedがあったら: `random.seed`を設定することでUser毎に一意のシード値で固定することで出力を固定する。
- seedがなかったら: 完全ランダム
- User id(0~)
- n: 再送回数
- slots: スロット数defaultは0
- transmitted: 送信成功回数defaultは0
- CW: Contention Window, calculate_CW関数の返り値が代入される(関数内で決定されたslotsにSLOT_TIMEをかけた値)
- total_data_transmitted: 送信できたデータの総和

### calculate_CW
返り値: CW[s]

CWのスロットの最大値`cw_max`は2^(4 + 再送回数n) - 1なので,そのユーザーのスロットの数`self.slots`は

1～min(`cw_max`, 1023)の間のランダムの値で決まり,代入する.

`self.slots`に`SLOT_TIME`をかけてCWを返している

### re_transmit
再送処理

`self.n += 1`で再送回数をインクリメントした状態でcalculate_CW関数を呼ぶことで前述の`cw_max`が大きくなった状態でランダム関数を通る

### reset_CW
送信完了したら再送回数を0にしてCWを代入しなおし

In [ ]:
class User:
    # Initialization of User class
    def __init__(self, id, n=0, seed=None):
        if seed is not None:
            # Set random seed
            random.seed(seed + id)
        self.id = id
        self.n = n  # Number of retries (for CW extension)
        self.slots = 0  # Number of waiting slots
        self.transmitted = 0  # Number of successful transmissions
        self.CW = self.calculate_CW()  # Contention window
        self.total_data_transmitted = 0  # Total data transmitted

    # Calculate the Contention Window (CW)
    def calculate_CW(self):
        cw_max = 2**(4 + self.n) - 1  # Maximum value of CW
        self.slots = random.randint(1, min(cw_max, 1023))  # Limit number of slots to 1023
        return self.slots * SLOT_TIME

    # Update CW when retransmitting
    def re_transmit(self):
        self.n += 1
        self.CW = self.calculate_CW()

    # Reset CW after successful transmission
    def reset_CW(self):
        self.n = 0
        self.CW = self.calculate_CW()